In [1]:
# ==========================================================================
# Cell 1a: SETUP AND IMPORTS - COMPUTATION
# ==========================================================================
import sys
import time
import numpy as np
import warnings
from IPython.display import clear_output

clear_output(wait=True)
warnings.filterwarnings('ignore')

# Import testing utilities - provides all shared infrastructure
from qectostim.testing import (
    # Status indicators
    STATUS_OK, STATUS_WARN, STATUS_SKIP, STATUS_FAIL,
    # Module management
    clear_qectostim_modules,
    # Test function
    test_decoder_on_code,
    # Decoder/code loading utilities
    load_all_decoders,
    discover_and_categorize_codes,
    print_code_summary,
    # Results formatting
    format_status,
    compute_summary_stats,
)

# Clear module cache for fresh testing
clear_qectostim_modules()

# Discover and categorize all codes WITH FAILURE REPORTING
# NOTE: Bosonic (GKP, rotor), qudit (Galois), and fracton (XCube, Haah) codes
# are excluded by default as they don't work with standard qubit decoders
categories, all_codes, failures = discover_and_categorize_codes(
    max_qubits=100,
    include_qldpc=True,
    include_subsystem=True,
    include_floquet=True,
    include_bosonic=False,   # Exclude - continuous variables
    include_qudit=False,     # Exclude - d>2 dimensions
    include_fracton=False,   # Exclude - exotic excitations
    report_failures=True,    # Report codes that failed to instantiate
)

print(f"✓ Discovered {len(all_codes)} codes in {len(categories)} categories")

✓ Discovered 75 codes in 6 categories


In [2]:
# ==========================================================================
# Cell 1b: SETUP AND IMPORTS - DISPLAY
# ==========================================================================

# Print summary
print_code_summary(categories, "DECODER SMOKE TEST - ALL CODE TYPES")

# Show failed codes if any
if failures:
    print("\n" + "=" * 70)
    print("FAILED TO INSTANTIATE")
    print("=" * 70)
    print(f"\n{len(failures)} codes failed to load:\n")
    
    # Group by error type for readability
    skipped = {k: v for k, v in failures.items() if v.startswith("SKIPPED:")}
    timeout = {k: v for k, v in failures.items() if v.startswith("TIMEOUT:")}
    not_found = {k: v for k, v in failures.items() if v.startswith("NOT_FOUND:")}
    errors = {k: v for k, v in failures.items() 
              if not v.startswith(("SKIPPED:", "TIMEOUT:", "NOT_FOUND:"))}
    
    if errors:
        print(f"ERRORS ({len(errors)} codes - bugs to fix):")
        for name, error in sorted(errors.items()):
            print(f"  ❌ {name}: {error}")
    
    if not_found:
        print(f"\nNOT FOUND ({len(not_found)} codes):")
        for name in sorted(not_found.keys()):
            print(f"  ❓ {name}")
    
    if timeout:
        print(f"\nTIMEOUT ({len(timeout)} codes):")
        for name in sorted(timeout.keys()):
            print(f"  ⏱️ {name}")
    
    if skipped:
        print(f"\nSKIPPED ({len(skipped)} codes - known slow/special):")
        for name in sorted(skipped.keys()):
            print(f"  ⏭️ {name}")

DECODER SMOKE TEST - ALL CODE TYPES

Total discovered: 75 codes
  CSS: 47
  Color: 3
  Non-CSS: 5
  Subsystem: 4
  Floquet: 6
  QLDPC: 10

CSS Codes:
Code Name                                   n   k   d
-------------------------------------------------------
  FourQubit422_[[4,2,2]]                    4   2   2
  C6                                        6   2   2
  Steane_713                                7   1   3
  Shor_91                                   9   1   3
  ReedMuller_15_1_3                        15   1   3
  Hamming_CSS_7                             7   1   3
  Hamming_CSS_15                           15   7   3
  Hamming_CSS_31                           31  21   3
  Code_832                                  8   3   2
  Repetition_3                              3   1   3
  Repetition_5                              5   1   5
  Repetition_7                              7   1   7
  RotatedSurface_[[9,1,3]]                  9   1   3
  RotatedSurface_[[25,1,5]]           

In [3]:
# Cell 2: Load Available Decoders - Using testing_utils

decoder_classes = load_all_decoders()
print(f"Loaded {len(decoder_classes)} decoders: {list(decoder_classes.keys())}")

Loaded 13 decoders: ['PyMatching', 'FusionBlossom', 'BeliefMatching', 'BPOSD', 'Tesseract', 'UnionFind', 'MLE', 'Hypergraph', 'Chromobius', 'Concatenated', 'FlatConcat', 'Hierarchical', 'SingleShot']


In [4]:
# ==========================================================================
# Cell 3a: DECODER × CODE COMPATIBILITY MATRIX - COMPUTATION
# ==========================================================================
from IPython.display import clear_output
clear_output(wait=True)

p = 0.001
shots = 5000
rounds = 1

# Store results
full_results = {}
dec_names = list(decoder_classes.keys())

# Process each code
for code_name, (code_type, code) in all_codes.items():
    full_results[code_name] = {'type': code_type}
    
    for dec_name in dec_names:
        result = test_decoder_on_code(
            code=code,
            decoder_class=decoder_classes[dec_name],
            decoder_name=dec_name,
            code_type=code_type,
            p=p,
            shots=shots,
            rounds=rounds
        )
        full_results[code_name][dec_name] = result.to_dict()

print(f"✓ Tested {len(all_codes)} codes × {len(dec_names)} decoders = {len(all_codes) * len(dec_names)} combinations")

✓ Tested 75 codes × 13 decoders = 975 combinations


In [5]:
# ==========================================================================
# Cell 3b: DECODER × CODE COMPATIBILITY MATRIX - DISPLAY
# ==========================================================================

print("="*130)
print("DECODER × CODE COMPATIBILITY MATRIX - ALL CODE TYPES")
print("="*130)
print(f"\nTested {len(all_codes)} codes at p={p}, {shots} shots")
print("Code types: CSS, Non-CSS, Subsystem, Floquet, QLDPC, Color")
print("Note: Chromobius requires color-code-like DEMs (shows SKIP for other codes)\n")

# Header
header = f"{'Code':<35} {'Type':<10}"
for dec_name in dec_names:
    header += f" | {dec_name[:10]:^12}"
print(header)
print("-" * len(header))

# Display each code
for code_name, code_results in full_results.items():
    code_type = code_results.get('type', 'Unknown')
    row = f"{code_name:<35} {code_type:<10}"
    
    for dec_name in dec_names:
        res = code_results.get(dec_name, {})
        status = res.get('status') if isinstance(res, dict) else None
        ler = res.get('ler') if isinstance(res, dict) else None
        warnings_list = res.get('warnings', []) if isinstance(res, dict) else []
        
        if status == 'OK':
            if warnings_list:
                cell = f"⚠️{ler:.4f}" if ler is not None else "⚠️N/A"
            else:
                cell = f"✓ {ler:.4f}" if ler is not None else "✓ N/A"    
        elif status == 'WARN':
            cell = f"⚠️WARN"
        elif status == 'SKIP':
            cell = f"- SKIP"
        else:
            cell = f"✗ FAIL"
        
        row += f" | {cell:^12}"
    
    print(row)

print("-" * len(header))
print(f"\nLegend: ✓=pass, ⚠️=warning, -=skip (expected), ✗=fail")
print(f"Total codes tested: {len(full_results)}")

DECODER × CODE COMPATIBILITY MATRIX - ALL CODE TYPES

Tested 75 codes at p=0.001, 5000 shots
Code types: CSS, Non-CSS, Subsystem, Floquet, QLDPC, Color
Note: Chromobius requires color-code-like DEMs (shows SKIP for other codes)

Code                                Type       |  PyMatching  |  FusionBlos  |  BeliefMatc  |    BPOSD     |  Tesseract   |  UnionFind   |     MLE      |  Hypergraph  |  Chromobius  |  Concatenat  |  FlatConcat  |  Hierarchic  |  SingleShot 
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
FourQubit422_[[4,2,2]]              CSS        |   ✓ 0.0046   |   ✓ 0.0046   |   ✓ 0.0042   |   ✓ 0.0058   |   ✓ 0.0040   |   ✓ 0.0056   |   ✓ 0.0044   |   ✓ 0.0028   |    - SKIP    |    - SKIP    |    - SKIP    |    - SKIP    |    - SKIP   
C6                                  CSS      

In [6]:
# Cell 4: LER COMPARISON TABLE - ALL CODES
from IPython.display import clear_output
clear_output(wait=True)

print("="*140)
print("LER COMPARISON TABLE (p=0.01) - ALL CODE TYPES")
print("="*140)
print(f"\nTotal codes in all_codes: {len(all_codes)}")
print(f"Total codes in full_results: {len(full_results)}")
print("\nLower is better. Best decoder for each code highlighted.")
print("SKIP = decoder incompatible (e.g., Chromobius requires color-code DEMs)\n")

# Header
dec_names = list(decoder_classes.keys())
header = f"{'Code':<35} | {'Type':<10} | {'d':>2} | {'No-decode':>10}"
for dec_name in dec_names:
    header += f" | {dec_name[:10]:>10}"
header += " | Best"
print(header)
print("-" * len(header))

# Use full_results directly since it has all the data from Cell 4
for code_name in sorted(full_results.keys()):
    code_results = full_results[code_name]
    code_type = code_results.get('type', 'Unknown')
    
    # Get code object for distance
    code = all_codes.get(code_name, (None, None))[1]
    d = code.metadata.get('distance', '?') if code else '?'
    
    # Get no-decode LER
    ler_no_decode = None
    for key, res in code_results.items():
        if isinstance(res, dict) and res.get('ler_no_decode') is not None:
            ler_no_decode = res['ler_no_decode']
            break
    
    nd_str = f"{ler_no_decode:.4f}" if ler_no_decode else 'N/A'
    row = f"{code_name:<35} | {code_type:<10} | {str(d):>2} | {nd_str:>10}"
    
    # Find best decoder
    best_ler = float('inf')
    best_decoder = None
    
    for dec_name in dec_names:
        res = code_results.get(dec_name, {})
        status = res.get('status') if isinstance(res, dict) else None
        ler = res.get('ler') if isinstance(res, dict) else None
        
        if status == 'SKIP':
            ler_str = 'SKIP'
        elif ler is not None:
            ler_str = f"{ler:.4f}"
            if ler < best_ler:
                best_ler = ler
                best_decoder = dec_name
        else:
            ler_str = 'FAIL'
        row += f" | {ler_str:>10}"
    
    best_str = best_decoder[:10] if best_decoder else 'N/A'
    row += f" | {best_str}"
    print(row)

print("-" * len(header))
print(f"\nTotal rows: {len(full_results)}")

LER COMPARISON TABLE (p=0.01) - ALL CODE TYPES

Total codes in all_codes: 75
Total codes in full_results: 75

Lower is better. Best decoder for each code highlighted.
SKIP = decoder incompatible (e.g., Chromobius requires color-code DEMs)

Code                                | Type       |  d |  No-decode | PyMatching | FusionBlos | BeliefMatc |      BPOSD |  Tesseract |  UnionFind |        MLE | Hypergraph | Chromobius | Concatenat | FlatConcat | Hierarchic | SingleShot | Best
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
BB_6x6                              | QLDPC      |  ? |     0.0186 |       FAIL |     0.0158 |       SKIP |     0.0000 |     0.0000 |       FAIL |       SKIP |       FAIL |       SKIP |       SKIP |       SKIP |       SKIP |       SKIP | BPOSD
BaconShor_3x3                 

In [7]:
# Cell to identify codes with WARN or FAIL status
print("="*80)
print("CODES WITH WARN OR FAIL STATUS")
print("="*80)

problem_codes = {}
for code_name, code_results in full_results.items():
    issues = []
    for dec_name in decoder_classes.keys():
        res = code_results.get(dec_name, {})
        status = res.get('status') if isinstance(res, dict) else None
        if status in ('WARN', 'FAIL'):
            issues.append((dec_name, status, res.get('warnings', []), res.get('error', '')))
    if issues:
        problem_codes[code_name] = issues

print(f"\nTotal codes with issues: {len(problem_codes)}\n")
for code_name, issues in sorted(problem_codes.items()):
    print(f"\n{code_name}:")
    # Count unique statuses
    warns = sum(1 for i in issues if i[1] == 'WARN')
    fails = sum(1 for i in issues if i[1] == 'FAIL')
    print(f"  WARN: {warns}, FAIL: {fails}")
    # Show first warning for context
    for dec, status, warnings, error in issues[:2]:
        if warnings:
            print(f"    {dec}: {warnings[0][:80]}...")
        elif error:
            print(f"    {dec}: {error[:80]}...")
        break

CODES WITH WARN OR FAIL STATUS

Total codes with issues: 3


BB_6x6:
  WARN: 3, FAIL: 0
    PyMatching: Decoder incompatibility...

ToricCode3DFaces_3x3x3:
  WARN: 3, FAIL: 0
    PyMatching: Decoder incompatibility...

ToricCode4D_2:
  WARN: 3, FAIL: 0
    PyMatching: Decoder incompatibility...


In [8]:
# Final full smoke test with fresh imports
import sys

# Remove all cached qectostim modules
modules_to_remove = [key for key in sys.modules.keys() if 'qectostim' in key]
for mod in modules_to_remove:
    del sys.modules[mod]

import warnings
from qectostim.codes.surface.toric_code_general import ToricCode
from qectostim.codes.surface import RotatedSurfaceCode
from qectostim.codes.small import RepetitionCode, SteaneCode713, ShorCode91, EightThreeTwoCode
from qectostim.codes.surface import ToricCode4D
from qectostim.codes.color import TriangularColourCode, HexagonalColourCode, ColorCode3D
from qectostim.codes.color.extended_color import TetrahedralColorCode
from qectostim.experiments import CSSMemoryExperiment
import stim

# Test codes
test_codes = [
    ("RotatedSurface_d3", RotatedSurfaceCode(3)),
    ("ToricCode_3x3", ToricCode(3)),
    ("ToricCode_5x5", ToricCode(5)),
    ("RepetitionCode_5", RepetitionCode(5)),
    ("SteaneCode", SteaneCode713()),
    ("ShorCode", ShorCode91()),
    ("Code_832", EightThreeTwoCode()),
    ("ToricCode4D_2", ToricCode4D(2)),
    ("TriangularColour_d5", TriangularColourCode(distance=5)),
    ("HexagonalColour_d3", HexagonalColourCode(distance=3)),
    ("ColorCode3D_d3", ColorCode3D(distance=3)),
    ("TetrahedralColorCode", TetrahedralColorCode()),
]

print("="*80)
print("FINAL Smoke Test Results (all fixes applied)")
print("="*80)

results = {"pass": [], "fail": []}

for name, code in test_codes:
    try:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            exp = CSSMemoryExperiment(code, rounds=2, basis="Z")
            circuit = exp.to_stim()
            
            # Try to build DEM
            dem = circuit.detector_error_model(decompose_errors=True)
            n_det = dem.num_detectors
            n_obs = dem.num_observables
            results["pass"].append(name)
            print(f"✓ {name}: PASS (det={n_det}, obs={n_obs})")
                
    except Exception as e:
        err_msg = str(e)
        if "non-deterministic" in err_msg:
            results["fail"].append((name, "Non-deterministic observables"))
        else:
            results["fail"].append((name, str(e)[:60]))
        print(f"✗ {name}: {str(e)[:80]}")

print("\n" + "="*80)
print("Summary")
print("="*80)
print(f"PASS: {len(results['pass'])}/{len(test_codes)}")
print(f"FAIL: {len(results['fail'])}/{len(test_codes)}")

if results["fail"]:
    print("\nFailed codes:")
    for name, msg in results["fail"]:
        print(f"  - {name}: {msg}")
else:
    print("\n🎉 ALL CODES PASS! 🎉")

FINAL Smoke Test Results (all fixes applied)
✓ RotatedSurface_d3: PASS (det=16, obs=1)
✓ ToricCode_3x3: PASS (det=32, obs=1)
✓ ToricCode_3x3: PASS (det=32, obs=1)
✓ ToricCode_3x3: PASS (det=32, obs=1)
✓ ToricCode_5x5: PASS (det=96, obs=1)
✓ RepetitionCode_5: PASS (det=12, obs=1)
✓ SteaneCode: PASS (det=12, obs=1)
✓ ShorCode: PASS (det=20, obs=1)
✓ ToricCode_5x5: PASS (det=96, obs=1)
✓ RepetitionCode_5: PASS (det=12, obs=1)
✓ SteaneCode: PASS (det=12, obs=1)
✓ ShorCode: PASS (det=20, obs=1)
✓ ToricCode_5x5: PASS (det=96, obs=1)
✓ RepetitionCode_5: PASS (det=12, obs=1)
✓ SteaneCode: PASS (det=12, obs=1)
✓ ShorCode: PASS (det=20, obs=1)
✓ Code_832: PASS (det=13, obs=1)
✓ Code_832: PASS (det=13, obs=1)
✓ Code_832: PASS (det=13, obs=1)
✓ ToricCode4D_2: PASS (det=256, obs=1)
✓ ToricCode4D_2: PASS (det=256, obs=1)
✓ ToricCode4D_2: PASS (det=256, obs=1)
✓ TriangularColour_d5: PASS (det=36, obs=1)
✓ HexagonalColour_d3: PASS (det=20, obs=1)
✓ TriangularColour_d5: PASS (det=36, obs=1)
✓ Hexagonal

In [9]:
# ==========================================================================
# Cell 5a: STIM REFERENCE VALIDATION - COMPUTATION
# ==========================================================================
"""
Compare our implementation vs Stim's built-in Circuit.generated()

Available Stim generated circuits:
- surface_code:rotated_memory_z / rotated_memory_x - Rotated surface codes
- repetition_code:memory - Repetition codes
- color_code:memory_xyz - ONLY XYZ color code (joint measurements with C_XYZ gates)

NOTE: Stim does NOT have a CSS color code variant (memory_z/memory_x).
Our CSS color code (TriangularColourCode + ColorCodeMemoryExperiment + Chromobius)
has no direct Stim reference for comparison but works correctly with Chromobius.

For XYZ color codes:
- Use TriangularColourCodeXYZ + XYZColorCodeMemoryExperiment + PyMatching
- Chromobius CANNOT decode XYZ codes (requires separate X/Z stabilizers)
"""

import stim
import sys
import numpy as np

# Clear module cache to ensure fresh imports with latest fixes
for m in [m for m in sys.modules if m.startswith('qectostim')]:
    del sys.modules[m]

from qectostim.codes.surface import RotatedSurfaceCode
from qectostim.codes.small import RepetitionCode
from qectostim.codes.color import TriangularColourCodeXYZ
from qectostim.experiments.memory import CSSMemoryExperiment, XYZColorCodeMemoryExperiment
from qectostim.noise.models import StimStyleDepolarizingNoise
from qectostim.decoders import PyMatchingDecoder, FusionBlossomDecoder

# QUICK_MODE: Set to True for fast CI testing, False for comprehensive testing
QUICK_MODE = True  # Fast validation for smoke test

# Configuration
if QUICK_MODE:
    STIM_SHOTS = 2000
    STIM_ROUNDS = 5
    STIM_CODES = [
        ("surface_code:rotated_memory_z", [3, 5]),
        ("repetition_code:memory", [5, 7]),
        ("color_code:memory_xyz", [3, 5]),    # XYZ color code - PyMatching only!
    ]
    STIM_NOISE_LEVELS = [0.005, 0.01]
else:
    STIM_SHOTS = 10000
    STIM_ROUNDS = 7
    STIM_CODES = [
        ("surface_code:rotated_memory_z", [3, 5, 7]),
        ("surface_code:rotated_memory_x", [3, 5]),
        ("repetition_code:memory", [5, 7, 9]),
        ("color_code:memory_xyz", [3, 5, 7]), # XYZ color code - PyMatching only!
    ]
    STIM_NOISE_LEVELS = [0.001, 0.005, 0.01]

def get_our_code(stim_task: str, distance: int):
    """Return the appropriate code class for comparison with Stim."""
    if "surface_code:rotated" in stim_task:
        return RotatedSurfaceCode(distance=distance)
    elif "repetition_code" in stim_task:
        return RepetitionCode(N=distance)
    elif "color_code:memory_xyz" in stim_task:
        return TriangularColourCodeXYZ(distance=distance)
    return None

def get_stim_circuit(stim_task: str, distance: int, rounds: int, p: float):
    return stim.Circuit.generated(
        stim_task,
        distance=distance,
        rounds=rounds,
        after_clifford_depolarization=p,
    )

def get_our_circuit(code, rounds: int, p: float, basis: str = "Z", code_type: str = "css"):
    """Build our circuit for comparison."""
    noise = StimStyleDepolarizingNoise(p=p)
    if code_type == "color_xyz":
        exp = XYZColorCodeMemoryExperiment(code=code, rounds=rounds, noise_model=None, basis=basis)
    else:
        exp = CSSMemoryExperiment(code=code, rounds=rounds, noise_model=None, basis=basis)
    return noise.apply(exp.to_stim())

def compute_ler(circuit, decoder_class, shots: int, ignore_decomp: bool = False):
    try:
        if ignore_decomp:
            dem = circuit.detector_error_model(decompose_errors=True, ignore_decomposition_failures=True)
        else:
            dem = circuit.detector_error_model(decompose_errors=True)
        decoder = decoder_class(dem=dem)
        sampler = circuit.compile_detector_sampler()
        samples = sampler.sample(shots, append_observables=True)
        det_samples = samples[:, :circuit.num_detectors]
        obs_samples = samples[:, circuit.num_detectors:]
        predictions = decoder.decode_batch(det_samples)
        if len(predictions.shape) == 1:
            predictions = predictions.reshape(-1, 1)
        errors = np.any(predictions != obs_samples, axis=1)
        return float(np.mean(errors))
    except Exception as e:
        return None

def compute_no_decode_ler(circuit, shots: int):
    try:
        sampler = circuit.compile_detector_sampler()
        samples = sampler.sample(shots, append_observables=True)
        obs_samples = samples[:, circuit.num_detectors:]
        return float(np.mean(obs_samples))
    except:
        return None

# Run all comparisons and store results
print("Running Stim reference validation...")
validation_results = {}
validation_details = []  # Store detailed results for display

for stim_task, distances in STIM_CODES:
    basis = "X" if "_x" in stim_task else "Z"
    is_xyz_color = "color_code:memory_xyz" in stim_task
    
    if is_xyz_color:
        code_type = "color_xyz"
        decoders_to_test = [("PyMatching", PyMatchingDecoder)]
        ignore_decomp = True
    else:
        code_type = "css"
        decoders_to_test = [("PyMatching", PyMatchingDecoder), ("FusionBlossom", FusionBlossomDecoder)]
        ignore_decomp = False
    
    for distance in distances:
        for p in STIM_NOISE_LEVELS:
            result_row = {
                'stim_task': stim_task,
                'distance': distance,
                'p': p,
                'decoders': [],
            }
            
            try:
                stim_circuit = get_stim_circuit(stim_task, distance, STIM_ROUNDS, p)
                our_code = get_our_code(stim_task, distance)
                
                if our_code is None:
                    result_row['error'] = 'NO CODE'
                    validation_details.append(result_row)
                    continue
                
                our_circuit = get_our_circuit(our_code, STIM_ROUNDS, p, basis, code_type)
                
                # No-decode comparison
                stim_nd = compute_no_decode_ler(stim_circuit, STIM_SHOTS)
                our_nd = compute_no_decode_ler(our_circuit, STIM_SHOTS)
                result_row['stim_nd'] = stim_nd
                result_row['our_nd'] = our_nd
                result_row['nd_ratio'] = our_nd / stim_nd if stim_nd and our_nd and stim_nd > 0 else None
                
                # DEM error count comparison
                if ignore_decomp:
                    stim_dem = stim_circuit.detector_error_model(decompose_errors=True, ignore_decomposition_failures=True)
                    our_dem = our_circuit.detector_error_model(decompose_errors=True, ignore_decomposition_failures=True)
                else:
                    stim_dem = stim_circuit.detector_error_model(decompose_errors=True)
                    our_dem = our_circuit.detector_error_model(decompose_errors=True)
                
                stim_errs = len([e for e in stim_dem.flattened() if e.type == 'error'])
                our_errs = len([e for e in our_dem.flattened() if e.type == 'error'])
                result_row['stim_errs'] = stim_errs
                result_row['our_errs'] = our_errs
                result_row['dem_match'] = stim_errs == our_errs
                
                # Decoder comparison
                for dec_name, dec_class in decoders_to_test:
                    stim_ler = compute_ler(stim_circuit, dec_class, STIM_SHOTS, ignore_decomp)
                    our_ler = compute_ler(our_circuit, dec_class, STIM_SHOTS, ignore_decomp)
                    
                    dec_result = {'name': dec_name, 'stim_ler': stim_ler, 'our_ler': our_ler}
                    
                    if stim_ler is not None and our_ler is not None and stim_ler > 0:
                        ratio = our_ler / stim_ler
                        dec_result['ratio'] = ratio
                        key = (stim_task, distance, p, dec_name)
                        validation_results[key] = {'stim_ler': stim_ler, 'our_ler': our_ler, 'ratio': ratio}
                    elif stim_ler == 0 and our_ler == 0:
                        dec_result['ratio'] = 'BOTH_0'
                    else:
                        dec_result['ratio'] = None
                    
                    result_row['decoders'].append(dec_result)
                    
            except Exception as e:
                result_row['error'] = str(e)[:50]
            
            validation_details.append(result_row)

print(f"✓ Completed {len(validation_details)} test configurations")

Running Stim reference validation...
✓ Completed 12 test configurations
✓ Completed 12 test configurations
✓ Completed 12 test configurations


In [10]:
# ==========================================================================
# Cell 5b: STIM REFERENCE VALIDATION - DISPLAY
# ==========================================================================

# Pretty print validation results
print("=" * 80)
print("STIM REFERENCE VALIDATION RESULTS")
print("=" * 80)

for row in validation_details:
    task_short = row['stim_task'].replace("_code:", "/").replace("rotated_memory", "rot")
    print(f"\n{task_short} d={row['distance']} p={row['p']:.4f}")
    
    if 'error' in row:
        print(f"  ❌ ERROR: {row['error']}")
        continue
    
    # DEM comparison
    dem_icon = "✓" if row.get('dem_match') else "✗"
    print(f"  DEM errors: {row.get('our_errs', '?')}/{row.get('stim_errs', '?')} {dem_icon}")
    
    # Decoder results
    for dec in row.get('decoders', []):
        if dec.get('ratio') is not None and dec.get('ratio') not in ['BOTH_0', None]:
            ratio = dec['ratio']
            icon = "✓" if 0.5 < ratio < 2.0 else "?"
            print(f"  {dec['name']}: LER ratio={ratio:.3f} {icon}")
        elif dec.get('ratio') == 'BOTH_0':
            print(f"  {dec['name']}: LER=0 for both ✓")
        else:
            stim_ler = dec.get('stim_ler', 'None')
            our_ler = dec.get('our_ler', 'None')
            print(f"  {dec['name']}: Stim={stim_ler}, Ours={our_ler}")

# Summary statistics
if validation_results:
    ratios = [v['ratio'] for v in validation_results.values() if isinstance(v.get('ratio'), (int, float))]
    if ratios:
        print("\n" + "=" * 80)
        print("SUMMARY")
        print("=" * 80)
        print(f"  Tests completed: {len(validation_results)}")
        print(f"  Average LER ratio: {np.mean(ratios):.3f} ± {np.std(ratios):.3f}")
        print(f"  Range: {min(ratios):.3f} - {max(ratios):.3f}")
        within_2x = sum(1 for r in ratios if 0.5 < r < 2.0) / len(ratios) * 100
        print(f"  Within 2x: {within_2x:.0f}%")
        status = "GOOD" if np.mean(ratios) > 0.5 and np.mean(ratios) < 2.0 and within_2x > 80 else "CHECK"
        print(f"  Status: {status}")

STIM REFERENCE VALIDATION RESULTS

surface/rot_z d=3 p=0.0050
  DEM errors: 570/570 ✓
  PyMatching: LER ratio=1.636 ✓
  FusionBlossom: LER ratio=0.897 ✓

surface/rot_z d=3 p=0.0100
  DEM errors: 570/570 ✓
  PyMatching: LER ratio=1.085 ✓
  FusionBlossom: LER ratio=0.889 ✓

surface/rot_z d=5 p=0.0050
  DEM errors: 1953/1953 ✓
  PyMatching: LER ratio=0.455 ?
  FusionBlossom: LER ratio=1.013 ✓

surface/rot_z d=5 p=0.0100
  DEM errors: 1953/1953 ✓
  PyMatching: LER ratio=0.884 ✓
  FusionBlossom: LER ratio=0.979 ✓

repetition/memory d=5 p=0.0050
  DEM errors: 95/95 ✓
  PyMatching: LER=0 for both ✓
  FusionBlossom: LER=0 for both ✓

repetition/memory d=5 p=0.0100
  DEM errors: 95/95 ✓
  PyMatching: LER=0 for both ✓
  FusionBlossom: LER ratio=2.000 ?

repetition/memory d=7 p=0.0050
  DEM errors: 135/135 ✓
  PyMatching: LER=0 for both ✓
  FusionBlossom: LER=0 for both ✓

repetition/memory d=7 p=0.0100
  DEM errors: 135/135 ✓
  PyMatching: LER=0 for both ✓
  FusionBlossom: LER ratio=0.000 ?

col

In [11]:
# Cell 6: DECODER SPEED COMPARISON
from IPython.display import clear_output
clear_output(wait=True)

print("="*110)
print("DECODER SPEED COMPARISON (time in ms for 1000 shots)")
print("="*110)
print(f"\nTotal codes in full_results: {len(full_results)}\n")

# Header
dec_names = list(decoder_classes.keys())
header = f"{'Code':<30} {'Type':<10}"
for dec_name in dec_names:
    header += f" | {dec_name[:10]:>10}"
print(header)
print("-" * len(header))

# Use full_results directly since it has all the data from Cell 4
for code_name in sorted(full_results.keys()):
    code_results = full_results[code_name]
    code_type = code_results.get('type', 'Unknown')
    row = f"{code_name:<30} {code_type:<10}"
    
    for dec_name in dec_names:
        res = code_results.get(dec_name, {})
        time_ms = res.get('time_ms') if isinstance(res, dict) else None
        time_str = f"{time_ms:.1f}" if time_ms else 'FAIL'
        row += f" | {time_str:>10}"
    
    print(row)

print("-" * len(header))
print(f"\nTotal rows: {len(full_results)}")

DECODER SPEED COMPARISON (time in ms for 1000 shots)

Total codes in full_results: 75

Code                           Type       | PyMatching | FusionBlos | BeliefMatc |      BPOSD |  Tesseract |  UnionFind |        MLE | Hypergraph | Chromobius | Concatenat | FlatConcat | Hierarchic | SingleShot
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
BB_6x6                         QLDPC      |       FAIL |      200.0 |       FAIL |      709.2 |      140.4 |       FAIL |       FAIL |       FAIL |       FAIL |       FAIL |       FAIL |       FAIL |       FAIL
BaconShor_3x3                  Subsystem  |        0.1 |       18.6 |       20.8 |       45.5 |       46.6 |        0.1 |        1.5 |        2.1 |       FAIL |       FAIL |       FAIL |       FAIL |       FAIL
BalancedProduct_5x5_G1         QLDPC      |        6.2 |       42.5 |

In [12]:
# ==========================================================================
# Cell 7a: NOISE LEVEL SCALING TEST - COMPUTATION
# ==========================================================================

# Pick a representative CSS code (RotatedSurface d=3) 
test_code_name = None
test_code_type = None
for name, (ctype, code) in all_codes.items():
    if 'RotatedSurface' in name and code.metadata.get('distance') == 3:
        test_code_name = name
        test_code_type = ctype
        break

if test_code_name is None:
    test_code_name = list(all_codes.keys())[0]
    test_code_type = all_codes[test_code_name][0]

test_code = all_codes[test_code_name][1]
noise_levels = [0.001, 0.005, 0.01, 0.02]

# Store results
noise_scaling_results = {}

for p_val in noise_levels:
    noise_scaling_results[p_val] = {}
    ler_no_decode = None
    
    for dec_name in dec_names:
        result = test_decoder_on_code(
            code=test_code,
            code_type=test_code_type,
            decoder_name=dec_name,
            decoder_class=decoder_classes[dec_name],
            p=p_val,
            shots=2000,
            rounds=3
        )
        
        if ler_no_decode is None and result.ler_no_decode is not None:
            ler_no_decode = result.ler_no_decode
        
        noise_scaling_results[p_val][dec_name] = result.ler
    
    noise_scaling_results[p_val]['no_decode'] = ler_no_decode

print(f"✓ Tested {test_code_name} at {len(noise_levels)} noise levels")

✓ Tested RotatedSurface_[[9,1,3]] at 4 noise levels


In [13]:
# ==========================================================================
# Cell 7b: NOISE LEVEL SCALING TEST - DISPLAY
# ==========================================================================

print("="*110)
print("NOISE LEVEL SCALING TEST")
print("="*110)
print(f"\nTesting {test_code_name} ({test_code_type}) across noise levels with all decoders...\n")

# Header
header = f"{'p':<10}"
for dec_name in dec_names:
    header += f" | {dec_name[:12]:>12}"
header += " | No-decode"
print(header)
print("-" * len(header))

for p_val in noise_levels:
    row = f"{p_val:<10.4f}"
    
    for dec_name in dec_names:
        ler = noise_scaling_results[p_val].get(dec_name)
        ler_str = f"{ler:.6f}" if ler is not None else 'FAIL'
        row += f" | {ler_str:>12}"
    
    nd = noise_scaling_results[p_val].get('no_decode')
    nd_str = f"{nd:.6f}" if nd else 'N/A'
    row += f" | {nd_str}"
    print(row)

print("-" * len(header))

NOISE LEVEL SCALING TEST

Testing RotatedSurface_[[9,1,3]] (CSS) across noise levels with all decoders...

p          |   PyMatching | FusionBlosso | BeliefMatchi |        BPOSD |    Tesseract |    UnionFind |          MLE |   Hypergraph |   Chromobius | Concatenated |   FlatConcat | Hierarchical |   SingleShot | No-decode
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
0.0010     |     0.000500 |     0.009500 |     0.001500 |     0.002000 |     0.000500 |     0.000000 |     0.001500 |     0.019000 |         FAIL |         FAIL |         FAIL |         FAIL |         FAIL | 0.010500
0.0050     |     0.007500 |     0.061500 |     0.009000 |     0.010000 |     0.006500 |     0.006000 |     0.018500 |     0.081500 |         FAIL |         FAIL |         FAIL |         FAIL |         FAIL | 0.071500
0.0100     |     0.0220

In [14]:
# ==========================================================================
# Cell 8a: MULTI-CODE NOISE SCALING (Threshold Behavior) - COMPUTATION
# ==========================================================================
# Test multiple codes at multiple distances to demonstrate threshold behavior
# Below threshold: LER decreases with distance
# Above threshold: LER increases with distance

# Clear module cache to ensure fresh imports with latest TICK fixes
import sys
for m in [m for m in sys.modules if m.startswith('qectostim')]:
    del sys.modules[m]

from qectostim.codes.surface import RotatedSurfaceCode
from qectostim.codes.surface import ToricCode
from qectostim.codes.small import SteaneCode713, ShorCode91
from qectostim.experiments.memory import CSSMemoryExperiment
from qectostim.noise.models import CircuitDepolarizingNoise
from qectostim.decoders import PyMatchingDecoder

# QUICK_MODE inherited from Stim validation cell (or define here if running standalone)
try:
    QUICK_MODE
except NameError:
    QUICK_MODE = True

# Configuration based on mode
if QUICK_MODE:
    SCALING_SHOTS = 100_000
    SCALING_ROUNDS = 3
    SCALING_CODES = {
        "RotatedSurface": [3, 5],
        "Toric": [(3, 3), (5, 5)],
    }
    SCALING_NOISE = [0.002, 0.005, 0.01, 0.015]
else:
    SCALING_SHOTS = 300_000
    SCALING_ROUNDS = 5
    SCALING_CODES = {
        "RotatedSurface": [3, 5, 7],
        "Toric": [(3, 3), (5, 5), (7, 7)],
        "Steane": [7],
        "Shor": [9],
    }
    SCALING_NOISE = [0.001, 0.002, 0.005, 0.008, 0.01, 0.012, 0.015, 0.02]

def instantiate_code(code_family: str, param):
    """Instantiate a code by family name and parameter."""
    if code_family == "RotatedSurface":
        return RotatedSurfaceCode(distance=param), f"d={param}"
    elif code_family == "Toric":
        return ToricCode(Lx=param[0], Ly=param[1]), f"{param[0]}x{param[1]}"
    elif code_family == "Steane":
        return SteaneCode713(), "[[7,1,3]]"
    elif code_family == "Shor":
        return ShorCode91(), "[[9,1,3]]"
    else:
        return None, "Unknown"

def compute_ler_for_scaling(code, p: float, rounds: int, shots: int):
    """Compute LER for a code at given noise level."""
    try:
        noise = CircuitDepolarizingNoise(p1=p, p2=p)
        exp = CSSMemoryExperiment(code=code, rounds=rounds, noise_model=None)
        circuit = noise.apply(exp.to_stim())
        
        if circuit.num_detectors == 0:
            return None
        
        dem = circuit.detector_error_model(decompose_errors=True)
        decoder = PyMatchingDecoder(dem=dem)
        
        sampler = circuit.compile_detector_sampler()
        samples = sampler.sample(shots, append_observables=True)
        
        det_samples = samples[:, :circuit.num_detectors]
        obs_samples = samples[:, circuit.num_detectors:]
        
        predictions = decoder.decode_batch(det_samples)
        if len(predictions.shape) == 1:
            predictions = predictions.reshape(-1, 1)
        
        errors = np.any(predictions != obs_samples, axis=1)
        return float(np.mean(errors))
    except Exception as e:
        return None

# Store results for analysis
scaling_results = {}
scaling_trends = {}  # Store trends for display

for code_family, params in SCALING_CODES.items():
    scaling_trends[code_family] = []
    
    for p in SCALING_NOISE:
        lers = []
        for param in params:
            code, label = instantiate_code(code_family, param)
            ler = compute_ler_for_scaling(code, p, SCALING_ROUNDS, SCALING_SHOTS)
            lers.append(ler)
            if ler is not None:
                key = (code_family, label, p)
                scaling_results[key] = ler
        
        # Determine trend
        valid_lers = [l for l in lers if l is not None]
        if len(valid_lers) >= 2:
            if valid_lers[-1] < valid_lers[0] * 0.8:
                trend = "↓ (sub-thresh)"
            elif valid_lers[-1] > valid_lers[0] * 1.2:
                trend = "↑ (super-thresh)"
            else:
                trend = "~ (threshold)"
        else:
            trend = "N/A"
        
        scaling_trends[code_family].append((p, lers, trend))

print(f"✓ Computed scaling for {len(SCALING_CODES)} code families × {len(SCALING_NOISE)} noise levels")

✓ Computed scaling for 2 code families × 4 noise levels


In [15]:
# ==========================================================================
# Cell 8b: MULTI-CODE NOISE SCALING - DISPLAY
# ==========================================================================

print("="*130)
print("MULTI-CODE NOISE SCALING TEST - Threshold Behavior Analysis")
print("="*130)
print(f"\nMode: {'QUICK' if QUICK_MODE else 'FULL'}")
print(f"Shots: {SCALING_SHOTS}, Rounds: {SCALING_ROUNDS}")
print(f"Decoder: PyMatching")
print("\nExpected behavior:")
print("  - Below threshold: LER decreases as distance increases (↓ with d)")
print("  - Above threshold: LER increases as distance increases (↑ with d)")
print("  - At threshold: LER roughly constant with distance (~)")

for code_family, params in SCALING_CODES.items():
    print(f"\n{'='*100}")
    print(f"Code Family: {code_family}")
    print(f"{'='*100}")
    
    # Header
    code_labels = [instantiate_code(code_family, param)[1] for param in params]
    header = f"{'p':<10}"
    for label in code_labels:
        header += f" | {label:>12}"
    header += " | Trend"
    print(header)
    print("-" * len(header))
    
    for p, lers, trend in scaling_trends[code_family]:
        row = f"{p:<10.4f}"
        for ler in lers:
            if ler is not None:
                row += f" | {ler:>12.6f}"
            else:
                row += f" | {'FAIL':>12}"
        row += f" | {trend}"
        print(row)

# Summary analysis
print("\n" + "="*130)
print("THRESHOLD ANALYSIS SUMMARY")
print("="*130)

for code_family, params in SCALING_CODES.items():
    if len(params) < 2:
        continue
        
    print(f"\n{code_family}:")
    
    for i, p in enumerate(SCALING_NOISE[:-1]):
        p_next = SCALING_NOISE[i + 1]
        
        code_small, label_small = instantiate_code(code_family, params[0])
        code_large, label_large = instantiate_code(code_family, params[-1])
        
        key_small_p = (code_family, label_small, p)
        key_large_p = (code_family, label_large, p)
        key_small_pn = (code_family, label_small, p_next)
        key_large_pn = (code_family, label_large, p_next)
        
        ler_small_p = scaling_results.get(key_small_p)
        ler_large_p = scaling_results.get(key_large_p)
        ler_small_pn = scaling_results.get(key_small_pn)
        ler_large_pn = scaling_results.get(key_large_pn)
        
        if all(x is not None for x in [ler_small_p, ler_large_p, ler_small_pn, ler_large_pn]):
            sub_thresh_at_p = ler_large_p < ler_small_p
            sub_thresh_at_pn = ler_large_pn < ler_small_pn
            
            if sub_thresh_at_p and not sub_thresh_at_pn:
                print(f"  Threshold estimate: between p={p} and p={p_next}")
                break
    else:
        print(f"  Threshold not crossed in tested range")

print("\n" + "="*130)

MULTI-CODE NOISE SCALING TEST - Threshold Behavior Analysis

Mode: QUICK
Shots: 100000, Rounds: 3
Decoder: PyMatching

Expected behavior:
  - Below threshold: LER decreases as distance increases (↓ with d)
  - Above threshold: LER increases as distance increases (↑ with d)
  - At threshold: LER roughly constant with distance (~)

Code Family: RotatedSurface
p          |          d=3 |          d=5 | Trend
------------------------------------------------
0.0020     |     0.001400 |     0.000110 | ↓ (sub-thresh)
0.0050     |     0.007190 |     0.002650 | ↓ (sub-thresh)
0.0100     |     0.024490 |     0.016720 | ↓ (sub-thresh)
0.0150     |     0.050760 |     0.048440 | ~ (threshold)

Code Family: Toric
p          |          3x3 |          5x5 | Trend
------------------------------------------------
0.0020     |     0.001210 |     0.000040 | ↓ (sub-thresh)
0.0050     |     0.007280 |     0.001270 | ↓ (sub-thresh)
0.0100     |     0.033100 |     0.013400 | ↓ (sub-thresh)
0.0150     |     0.

In [16]:
# ==========================================================================
# Cell 9a: SUB-THRESHOLD ROUNDS SCALING MATRIX - COMPUTATION
# ==========================================================================
# Test Code × Decoder compatibility at low noise with increasing rounds
# Expected: Below threshold, more rounds → lower LER (error correction working!)

# Clear module cache
import sys
for m in [m for m in sys.modules if m.startswith('qectostim')]:
    del sys.modules[m]

from qectostim.codes.surface import RotatedSurfaceCode
from qectostim.codes.surface import ToricCode
from qectostim.codes.small import SteaneCode713, ShorCode91, FourQubit422Code
from qectostim.experiments.memory import CSSMemoryExperiment
from qectostim.noise.models import CircuitDepolarizingNoise

try:
    QUICK_MODE
except NameError:
    QUICK_MODE = True

# Configuration
if QUICK_MODE:
    ROUNDS_SHOTS = 100_000
    SUB_THRESH_P = 0.003
    ROUNDS_LIST = [3, 5, 7]
    ROUNDS_CODES = {
        "FourQubit422": FourQubit422Code(),
        "Steane713": SteaneCode713(),
        "RotatedSurface_d3": RotatedSurfaceCode(distance=3),
        "RotatedSurface_d5": RotatedSurfaceCode(distance=5),
    }
else:
    ROUNDS_SHOTS = 200_000
    SUB_THRESH_P = 0.003
    ROUNDS_LIST = [3, 5, 7, 11]
    ROUNDS_CODES = {
        "FourQubit422": FourQubit422Code(),
        "Steane713": SteaneCode713(),
        "Shor91": ShorCode91(),
        "RotatedSurface_d3": RotatedSurfaceCode(distance=3),
        "RotatedSurface_d5": RotatedSurfaceCode(distance=5),
        "Toric_3x3": ToricCode(Lx=3, Ly=3),
        "Toric_5x5": ToricCode(Lx=5, Ly=5),
    }

# Get decoders
try:
    decoder_classes
except NameError:
    from qectostim.testing import load_all_decoders
    decoder_classes = load_all_decoders()

ROUNDS_DECODERS = ['PyMatching', 'FusionBlossom']
available_decoders = {k: v for k, v in decoder_classes.items() if k in ROUNDS_DECODERS}

def test_rounds_scaling(code, decoder_class, decoder_name: str, rounds_list: list, p: float, shots: int):
    """Test LER at multiple round counts."""
    results = {}
    for rounds in rounds_list:
        try:
            noise = CircuitDepolarizingNoise(p1=p, p2=p)
            exp = CSSMemoryExperiment(code=code, rounds=rounds, noise_model=None)
            circuit = noise.apply(exp.to_stim())
            
            if circuit.num_detectors == 0:
                results[rounds] = None
                continue
            
            dem = circuit.detector_error_model(decompose_errors=True)
            decoder = decoder_class(dem=dem)
            
            sampler = circuit.compile_detector_sampler()
            samples = sampler.sample(shots, append_observables=True)
            
            det_samples = samples[:, :circuit.num_detectors]
            obs_samples = samples[:, circuit.num_detectors:]
            
            predictions = decoder.decode_batch(det_samples)
            if len(predictions.shape) == 1:
                predictions = predictions.reshape(-1, 1)
            
            errors = np.any(predictions != obs_samples, axis=1)
            results[rounds] = float(np.mean(errors))
        except Exception as e:
            results[rounds] = None
    
    return results

def analyze_trend(lers: dict):
    """Analyze trend in LER vs rounds."""
    valid = [(r, l) for r, l in sorted(lers.items()) if l is not None]
    if len(valid) < 2:
        return "N/A", None
    
    first_ler = valid[0][1]
    last_ler = valid[-1][1]
    
    if last_ler < first_ler * 0.7:
        return "✓↓", last_ler / first_ler if first_ler > 0 else None
    elif last_ler > first_ler * 1.3:
        return "⚠↑", last_ler / first_ler if first_ler > 0 else None
    else:
        return "→", last_ler / first_ler if first_ler > 0 else None

# Store all results
rounds_results = {}

for dec_name in ROUNDS_DECODERS:
    if dec_name not in available_decoders:
        continue
    dec_class = available_decoders[dec_name]
    
    for code_name, code in ROUNDS_CODES.items():
        lers = test_rounds_scaling(code, dec_class, dec_name, ROUNDS_LIST, SUB_THRESH_P, ROUNDS_SHOTS)
        rounds_results[(code_name, dec_name)] = lers

print(f"✓ Tested {len(ROUNDS_CODES)} codes × {len(ROUNDS_DECODERS)} decoders × {len(ROUNDS_LIST)} rounds")

✓ Tested 4 codes × 2 decoders × 3 rounds


In [17]:
# ==========================================================================
# Cell 9b: SUB-THRESHOLD ROUNDS SCALING MATRIX - DISPLAY
# ==========================================================================

print("="*140)
print("SUB-THRESHOLD ROUNDS SCALING MATRIX")
print("="*140)
print(f"\nMode: {'QUICK' if QUICK_MODE else 'FULL'}")
print(f"Physical error rate: p = {SUB_THRESH_P} (sub-threshold for most codes)")
print(f"Rounds tested: {ROUNDS_LIST}")
print(f"Shots per test: {ROUNDS_SHOTS}")
print("\nExpected behavior: Below threshold, more rounds → LOWER LER (error correction succeeds!)")
print("  ✓↓ = LER decreasing with rounds (healthy sub-threshold behavior)")
print("  ⚠↑ = LER increasing with rounds (above threshold or bug)")
print("  →  = LER flat (threshold region)")
print()

for dec_name in ROUNDS_DECODERS:
    if dec_name not in available_decoders:
        continue
        
    print(f"\n{'='*100}")
    print(f"Decoder: {dec_name}")
    print(f"{'='*100}")
    
    # Header
    header = f"{'Code':<25}"
    for r in ROUNDS_LIST:
        header += f" | R={r:>3}"
    header += " | Trend | Ratio"
    print(header)
    print("-" * len(header))
    
    for code_name in ROUNDS_CODES.keys():
        row = f"{code_name:<25}"
        lers = rounds_results.get((code_name, dec_name), {})
        
        for r in ROUNDS_LIST:
            ler = lers.get(r)
            if ler is not None:
                row += f" | {ler:>6.4f}"
            else:
                row += f" | {'FAIL':>6}"
        
        trend, ratio = analyze_trend(lers)
        ratio_str = f"{ratio:.2f}" if ratio is not None else "N/A"
        row += f" | {trend:^5} | {ratio_str:>5}"
        
        print(row)
    
    print("-" * len(header))

# Summary
print("\n" + "="*140)
print("ROUNDS SCALING SUMMARY")
print("="*140)

for dec_name in ROUNDS_DECODERS:
    if dec_name not in available_decoders:
        continue
        
    improving = worsening = flat = failed = 0
    
    for code_name in ROUNDS_CODES.keys():
        lers = rounds_results.get((code_name, dec_name), {})
        trend, _ = analyze_trend(lers)
        
        if trend == "✓↓":
            improving += 1
        elif trend == "⚠↑":
            worsening += 1
        elif trend == "→":
            flat += 1
        else:
            failed += 1
    
    total = improving + worsening + flat + failed
    print(f"\n{dec_name}:")
    print(f"  ✓↓ Improving (sub-threshold): {improving}/{total}")
    print(f"  →  Flat (threshold region):   {flat}/{total}")
    print(f"  ⚠↑ Worsening (super-thresh):  {worsening}/{total}")
    print(f"  N/A Failed:                   {failed}/{total}")
    
    if worsening > 0:
        print(f"  ⚠️ WARNING: {worsening} codes showing above-threshold behavior at p={SUB_THRESH_P}")
    elif improving == total - failed:
        print(f"  ✓ All codes showing healthy sub-threshold behavior!")

print("\n" + "="*140)

SUB-THRESHOLD ROUNDS SCALING MATRIX

Mode: QUICK
Physical error rate: p = 0.003 (sub-threshold for most codes)
Rounds tested: [3, 5, 7]
Shots per test: 100000

Expected behavior: Below threshold, more rounds → LOWER LER (error correction succeeds!)
  ✓↓ = LER decreasing with rounds (healthy sub-threshold behavior)
  ⚠↑ = LER increasing with rounds (above threshold or bug)
  →  = LER flat (threshold region)


Decoder: PyMatching
Code                      | R=  3 | R=  5 | R=  7 | Trend | Ratio
-----------------------------------------------------------------
FourQubit422              | 0.0293 | 0.0437 | 0.0587 |  ⚠↑   |  2.00
Steane713                 | 0.0417 | 0.0678 | 0.0903 |  ⚠↑   |  2.16
RotatedSurface_d3         | 0.0024 | 0.0046 | 0.0058 |  ⚠↑   |  2.43
RotatedSurface_d5         | 0.0005 | 0.0010 | 0.0014 |  ⚠↑   |  2.65
-----------------------------------------------------------------

Decoder: FusionBlossom
Code                      | R=  3 | R=  5 | R=  7 | Trend | Ratio
----

In [18]:
# ==========================================================================
# Cell 10a: SURFACE CODE THRESHOLD SCALING - COMPUTATION
# ==========================================================================
import numpy as np
import stim
import time
import sys

for mod_name in list(sys.modules.keys()):
    if 'qectostim' in mod_name:
        del sys.modules[mod_name]

from qectostim.codes.surface.rotated_surface import RotatedSurfaceCode
from qectostim.codes.color.triangular_colour import TriangularColourCode
from qectostim.experiments.memory import CSSMemoryExperiment, ColorCodeMemoryExperiment
from qectostim.noise.models import CircuitDepolarizingNoise
from qectostim.decoders.decoder_selector import select_decoder

distances = [3, 5, 7]
p_values = [0.001, 0.005, 0.01]
shots = 1000

results_surface = {}

for d in distances:
    code = RotatedSurfaceCode(distance=d)
    results_surface[d] = {}
    
    for p in p_values:
        noise = CircuitDepolarizingNoise(p1=p, p2=p)
        exp = CSSMemoryExperiment(code=code, noise_model=noise, rounds=d, basis="Z")
        
        start = time.time()
        circuit = noise.apply(exp.to_stim())
        dem = circuit.detector_error_model(decompose_errors=True)
        decoder = select_decoder(dem, preferred="pymatching")
        
        sampler = dem.compile_sampler()
        raw = sampler.sample(shots=shots)
        det_samples = np.asarray(raw[0], dtype=np.uint8)
        obs_samples = np.asarray(raw[1], dtype=np.uint8)
        
        corrections = decoder.decode_batch(det_samples)
        corrections = np.asarray(corrections, dtype=np.uint8).reshape(-1, dem.num_observables)
        
        logical_errors = (corrections[:, 0] ^ obs_samples[:, 0]).astype(np.uint8)
        ler = float(logical_errors.mean())
        elapsed = time.time() - start
        
        results_surface[d][p] = ler

print(f"✓ Computed surface code LER for {len(distances)} distances × {len(p_values)} noise levels")

✓ Computed surface code LER for 3 distances × 3 noise levels


In [19]:
# ==========================================================================
# Cell 10b: SURFACE CODE THRESHOLD SCALING - DISPLAY
# ==========================================================================

print("=" * 70)
print("THRESHOLD SCALING: SURFACE CODES")
print("=" * 70)

for d in distances:
    for p in p_values:
        print(f"d={d}, p={p:.3f}: LER = {results_surface[d][p]:.4f}")

print("\n" + "-" * 50)
print("SURFACE CODE THRESHOLD SUMMARY")
print("-" * 50)
print(f"{'p':>8} | " + " | ".join(f"d={d:2d}" for d in distances))
print("-" * 50)
for p in p_values:
    row = f"{p:>8.3f} | " + " | ".join(f"{results_surface[d].get(p, float('nan')):.4f}" for d in distances)
    print(row)

# Threshold analysis
p_low = 0.001
lers = [results_surface[d][p_low] for d in distances]
print(f"\nAt p={p_low}:")
arrow1 = '↓' if lers[0] > lers[1] else ('→' if lers[0] == lers[1] else '↑')
arrow2 = '↓' if lers[1] > lers[2] else ('→' if lers[1] == lers[2] else '↑')
print(f"  d=3→d=5: {lers[0]:.4f} → {lers[1]:.4f} {arrow1}")
print(f"  d=5→d=7: {lers[1]:.4f} → {lers[2]:.4f} {arrow2}")

THRESHOLD SCALING: SURFACE CODES
d=3, p=0.001: LER = 0.0010
d=3, p=0.005: LER = 0.0050
d=3, p=0.010: LER = 0.0240
d=5, p=0.001: LER = 0.0000
d=5, p=0.005: LER = 0.0060
d=5, p=0.010: LER = 0.0220
d=7, p=0.001: LER = 0.0000
d=7, p=0.005: LER = 0.0040
d=7, p=0.010: LER = 0.0260

--------------------------------------------------
SURFACE CODE THRESHOLD SUMMARY
--------------------------------------------------
       p | d= 3 | d= 5 | d= 7
--------------------------------------------------
   0.001 | 0.0010 | 0.0000 | 0.0000
   0.005 | 0.0050 | 0.0060 | 0.0040
   0.010 | 0.0240 | 0.0220 | 0.0260

At p=0.001:
  d=3→d=5: 0.0010 → 0.0000 ↓
  d=5→d=7: 0.0000 → 0.0000 →


In [20]:
# ==========================================================================
# Cell 11a: COLOR CODE THRESHOLD SCALING - COMPUTATION
# ==========================================================================

results_color = {}

for d in distances:
    code = TriangularColourCode(distance=d)
    results_color[d] = {}
    
    for p in p_values:
        noise = CircuitDepolarizingNoise(p1=p, p2=p)
        exp = ColorCodeMemoryExperiment(code=code, noise_model=noise, rounds=d, basis="Z")
        
        start = time.time()
        circuit = noise.apply(exp.to_stim())
        dem = circuit.detector_error_model(decompose_errors=True, ignore_decomposition_failures=True)
        decoder = select_decoder(dem, preferred="chromobius")
        
        sampler = dem.compile_sampler()
        raw = sampler.sample(shots=shots)
        det_samples = np.asarray(raw[0], dtype=np.uint8)
        obs_samples = np.asarray(raw[1], dtype=np.uint8)
        
        corrections = decoder.decode_batch(det_samples)
        corrections = np.asarray(corrections, dtype=np.uint8).reshape(-1, dem.num_observables)
        
        logical_errors = (corrections[:, 0] ^ obs_samples[:, 0]).astype(np.uint8)
        ler = float(logical_errors.mean())
        elapsed = time.time() - start
        
        results_color[d][p] = ler

print(f"✓ Computed color code LER for {len(distances)} distances × {len(p_values)} noise levels")

✓ Computed color code LER for 3 distances × 3 noise levels


In [21]:
# ==========================================================================
# Cell 11b: COLOR CODE THRESHOLD SCALING - DISPLAY
# ==========================================================================

print("=" * 70)
print("THRESHOLD SCALING: COLOR CODES (Chromobius)")
print("=" * 70)

for d in distances:
    for p in p_values:
        print(f"d={d}, p={p:.3f}: LER = {results_color[d][p]:.4f}")

print("\n" + "-" * 50)
print("COLOR CODE THRESHOLD SUMMARY")
print("-" * 50)
print(f"{'p':>8} | " + " | ".join(f"d={d:2d}" for d in distances))
print("-" * 50)
for p in p_values:
    row = f"{p:>8.3f} | " + " | ".join(f"{results_color[d].get(p, float('nan')):.4f}" for d in distances)
    print(row)

# Threshold analysis
lers = [results_color[d][p_low] for d in distances]
print(f"\nAt p={p_low}:")
arrow1 = '↓' if lers[0] > lers[1] else ('→' if lers[0] == lers[1] else '↑')
arrow2 = '↓' if lers[1] > lers[2] else ('→' if lers[1] == lers[2] else '↑')
print(f"  d=3→d=5: {lers[0]:.4f} → {lers[1]:.4f} {arrow1}")
print(f"  d=5→d=7: {lers[1]:.4f} → {lers[2]:.4f} {arrow2}")

THRESHOLD SCALING: COLOR CODES (Chromobius)
d=3, p=0.001: LER = 0.0050
d=3, p=0.005: LER = 0.0310
d=3, p=0.010: LER = 0.0660
d=5, p=0.001: LER = 0.0010
d=5, p=0.005: LER = 0.0290
d=5, p=0.010: LER = 0.0920
d=7, p=0.001: LER = 0.0000
d=7, p=0.005: LER = 0.0200
d=7, p=0.010: LER = 0.1380

--------------------------------------------------
COLOR CODE THRESHOLD SUMMARY
--------------------------------------------------
       p | d= 3 | d= 5 | d= 7
--------------------------------------------------
   0.001 | 0.0050 | 0.0010 | 0.0000
   0.005 | 0.0310 | 0.0290 | 0.0200
   0.010 | 0.0660 | 0.0920 | 0.1380

At p=0.001:
  d=3→d=5: 0.0050 → 0.0010 ↓
  d=5→d=7: 0.0010 → 0.0000 ↓


In [22]:
# ==========================================================================
# COMBINED THRESHOLD COMPARISON: Surface vs Color
# ==========================================================================

print("=" * 70)
print("COMBINED THRESHOLD COMPARISON")
print("=" * 70)

print("\n" + "=" * 50)
print("SURFACE CODE (PyMatching)")
print("=" * 50)
print(f"{'p':>8} | " + " | ".join(f"d={d:2d}" for d in distances))
print("-" * 50)
for p in p_values:
    row = f"{p:>8.3f} | " + " | ".join(f"{results_surface[d].get(p, float('nan')):.4f}" for d in distances)
    print(row)

print("\n" + "=" * 50)
print("COLOR CODE (Chromobius)")
print("=" * 50)
print(f"{'p':>8} | " + " | ".join(f"d={d:2d}" for d in distances))
print("-" * 50)
for p in p_values:
    row = f"{p:>8.3f} | " + " | ".join(f"{results_color[d].get(p, float('nan')):.4f}" for d in distances)
    print(row)

# Threshold analysis
print("\n" + "=" * 50)
print("THRESHOLD BEHAVIOR ANALYSIS (p=0.001)")
print("=" * 50)

surface_lers = [results_surface[d][p_low] for d in distances]
color_lers = [results_color[d][p_low] for d in distances]

print(f"\nSurface Code: d=3→d=5→d=7")
print(f"  LER: {surface_lers[0]:.4f} → {surface_lers[1]:.4f} → {surface_lers[2]:.4f}")
decreasing_surface = surface_lers[0] >= surface_lers[1] >= surface_lers[2]
print(f"  Threshold behavior: {'✅ PASS' if decreasing_surface else '❌ FAIL'}")

print(f"\nColor Code: d=3→d=5→d=7")
print(f"  LER: {color_lers[0]:.4f} → {color_lers[1]:.4f} → {color_lers[2]:.4f}")
decreasing_color = color_lers[0] >= color_lers[1] >= color_lers[2]
print(f"  Threshold behavior: {'✅ PASS' if decreasing_color else '❌ FAIL'}")

if decreasing_surface and decreasing_color:
    print("\n" + "=" * 50)
    print("🎉 BOTH CODE FAMILIES EXHIBIT CORRECT THRESHOLD SCALING!")
    print("=" * 50)

COMBINED THRESHOLD COMPARISON

SURFACE CODE (PyMatching)
       p | d= 3 | d= 5 | d= 7
--------------------------------------------------
   0.001 | 0.0010 | 0.0000 | 0.0000
   0.005 | 0.0050 | 0.0060 | 0.0040
   0.010 | 0.0240 | 0.0220 | 0.0260

COLOR CODE (Chromobius)
       p | d= 3 | d= 5 | d= 7
--------------------------------------------------
   0.001 | 0.0050 | 0.0010 | 0.0000
   0.005 | 0.0310 | 0.0290 | 0.0200
   0.010 | 0.0660 | 0.0920 | 0.1380

THRESHOLD BEHAVIOR ANALYSIS (p=0.001)

Surface Code: d=3→d=5→d=7
  LER: 0.0010 → 0.0000 → 0.0000
  Threshold behavior: ✅ PASS

Color Code: d=3→d=5→d=7
  LER: 0.0050 → 0.0010 → 0.0000
  Threshold behavior: ✅ PASS

🎉 BOTH CODE FAMILIES EXHIBIT CORRECT THRESHOLD SCALING!


In [23]:
# Final comprehensive test of all code fixes
import sys
modules_to_remove = [key for key in sys.modules.keys() if 'qectostim' in key]
for mod in modules_to_remove:
    del sys.modules[mod]

from qectostim.testing.testing_utils import categorize_codes
from qectostim.experiments.memory import CSSMemoryExperiment
from qectostim.noise.models import CircuitDepolarizingNoise

noise = CircuitDepolarizingNoise(p1=0.001, p2=0.001)

print("="*70)
print("COMPREHENSIVE CODE TEST - CHECKING ALL DISCOVERED CODES")
print("="*70)

# Get all codes
all_codes = categorize_codes(include_floquet=False)

# Count codes by category
total_css = len(all_codes.get('CSS', {}))
total_color = len(all_codes.get('Color', {}))
total_other = len(all_codes.get('Non-CSS', {})) + len(all_codes.get('Subsystem', {})) + len(all_codes.get('QLDPC', {}))

print(f"\nDiscovered codes:")
print(f"  CSS: {total_css}")
print(f"  Color: {total_color}")
print(f"  Other: {total_other}")

# Test a sample of codes
test_codes = []

# Add CSS codes
for name, code in list(all_codes.get('CSS', {}).items())[:15]:
    test_codes.append(('CSS', name, code))

# Add Color codes  
for name, code in list(all_codes.get('Color', {}).items())[:5]:
    test_codes.append(('Color', name, code))

print(f"\nTesting {len(test_codes)} codes...")

results = {'pass': 0, 'skip': 0, 'fail': 0}
failures = []
skips = []

for category, name, code in test_codes:
    skip = code._metadata.get('skip_standard_test', False)
    
    if skip:
        results['skip'] += 1
        skips.append(name)
        continue
        
    try:
        expt = CSSMemoryExperiment(code=code, noise_model=noise, rounds=2)
        circuit = expt.to_stim()
        dem = circuit.detector_error_model()
        results['pass'] += 1
    except Exception as e:
        results['fail'] += 1
        failures.append((name, str(e)[:60]))

print(f"\n{'='*70}")
print("RESULTS")
print("="*70)
print(f"Pass: {results['pass']}")
print(f"Skip: {results['skip']}")
print(f"Fail: {results['fail']}")

if skips:
    print(f"\nSkipped codes ({len(skips)}):")
    for name in skips[:5]:
        print(f"  - {name}")
    if len(skips) > 5:
        print(f"  ... and {len(skips)-5} more")

if failures:
    print(f"\nFailed codes ({len(failures)}):")
    for name, error in failures[:10]:
        print(f"  - {name}: {error}")

COMPREHENSIVE CODE TEST - CHECKING ALL DISCOVERED CODES


TypeError: categorize_codes() got an unexpected keyword argument 'include_floquet'